In [1]:
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import pymongo
import os

/home/ubuntu/.local/lib/python3.6/site-packages/pymongo/pyopenssl_context.py:26: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography. The next release of cryptography will remove support for Python 3.6.
  from cryptography.x509 import load_der_x509_certificate as _load_der_x509_certificate


In [2]:
# os.chdir('bogota-apartments/notebooks') # Only for server execution

In [3]:
load_dotenv()

True

In [4]:
client = pymongo.MongoClient(os.getenv('MONGO_URI'))
db = client[os.getenv('MONGO_DATABASE')]
collection = db['scrapy_bogota_apartments']

df = pd.DataFrame(list(collection.find()))
df = df.drop(columns=['_id'], axis=1)

df.head(3)

,codigo,tipo_propiedad,tipo_operacion,precio_venta,precio_arriendo,area,habitaciones,banos,administracion,parqueaderos,...,descripcion,datetime,website,compañia,imagenes,fecha_actualizacion_precio_venta,precio_venta_anterior,fecha_actualizacion_precio_arriendo,precio_arriendo_anterior,direccion
0,374-M2718950,APARTAMENTO,VENTA Y ARRIENDO,2.500000e+09,13500000.0,320.0,4.0,5.0,2500000.0,3.0,...,"vendo o arriendo ph duplex montearroyo, 320 mt...",2023-06-19 05:43:49.763,metrocuadrado.com,LIDIS MACIAS,[https://multimedia.metrocuadrado.com/374-M271...,NaT,NaN,NaT,NaN,NaN
1,2510-M4156145,APARTAMENTO,VENTA,4.850000e+08,NaN,84.0,3.0,3.0,680000.0,1.0,...,"oportunidad, bonito apartamento, recien refacc...",2023-06-19 05:43:53.411,metrocuadrado.com,JM GESTIONES INMOBILIARIAS,[https://multimedia.metrocuadrado.com/2510-M41...,NaT,NaN,NaT,NaN,NaN
2,10312-M2873283,APARTAMENTO,VENTA Y ARRIENDO,1.980000e+09,9000000.0,248.0,3.0,3.0,2200000.0,3.0,...,presentamos espectacular apartamento en uno de...,2023-06-19 05:43:58.765,metrocuadrado.com,PENTAGOLD,[https://multimedia.metrocuadrado.com/10312-M2...,NaT,NaN,NaT,NaN,NaN


In [6]:
df.columns

Index(['codigo', 'tipo_propiedad', 'tipo_operacion', 'precio_venta',
       'precio_arriendo', 'area', 'habitaciones', 'banos', 'administracion',
       'parqueaderos', 'sector', 'estrato', 'antiguedad', 'estado', 'longitud',
       'latitud', 'featured_interior', 'featured_exterior',
       'featured_zona_comun', 'featured_sector', 'descripcion', 'datetime',
       'website', 'compañia', 'imagenes', 'fecha_actualizacion_precio_venta',
       'precio_venta_anterior', 'fecha_actualizacion_precio_arriendo',
       'precio_arriendo_anterior', 'direccion'],
      dtype='object')

In [7]:
images_explode = df.explode('imagenes')
images_explode = images_explode.dropna(subset=['imagenes'])

In [8]:
images_df = images_explode[['codigo', 'imagenes']].rename(columns={'imagenes': 'url_imagen'})
images_df.shape

(986463, 2)

In [9]:
images_df.to_csv('../data/processed/images.csv', index=False)
images_df.columns

Index(['codigo', 'url_imagen'], dtype='object')

In [10]:
df = df.drop(columns=['imagenes'], axis=1)

In [11]:
from src import extract_features

df['jacuzzi'] = df.featured_interior.apply(extract_features.check_jacuzzi)
df['piso'] = df.featured_interior.apply(extract_features.extract_piso)
df['closets'] = df.featured_interior.apply(extract_features.extract_closets)
df['chimenea'] = df.featured_interior.apply(extract_features.check_chimeny)
df['permite_mascotas'] = df.featured_interior.apply(extract_features.check_mascotas)
df['gimnasio'] = df.featured_zona_comun.apply(extract_features.check_gimnasio)
df['ascensor'] = df.featured_exterior.apply(extract_features.check_ascensor)
df['conjunto_cerrado'] = df.featured_exterior.apply(extract_features.check_conjunto_cerrado)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65041 entries, 0 to 65040
Data columns (total 37 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   codigo                               65041 non-null  object        
 1   tipo_propiedad                       65041 non-null  object        
 2   tipo_operacion                       65041 non-null  object        
 3   precio_venta                         45734 non-null  float64       
 4   precio_arriendo                      20193 non-null  float64       
 5   area                                 65039 non-null  float64       
 6   habitaciones                         65033 non-null  float64       
 7   banos                                65035 non-null  float64       
 8   administracion                       52731 non-null  float64       
 9   parqueaderos                         65033 non-null  float64       
 10  sector    

In [13]:
df = df.drop(columns=['featured_interior', 'featured_zona_comun', 'featured_exterior', 'featured_sector'], axis=1)
df.columns

Index(['codigo', 'tipo_propiedad', 'tipo_operacion', 'precio_venta',
       'precio_arriendo', 'area', 'habitaciones', 'banos', 'administracion',
       'parqueaderos', 'sector', 'estrato', 'antiguedad', 'estado', 'longitud',
       'latitud', 'descripcion', 'datetime', 'website', 'compañia',
       'fecha_actualizacion_precio_venta', 'precio_venta_anterior',
       'fecha_actualizacion_precio_arriendo', 'precio_arriendo_anterior',
       'direccion', 'jacuzzi', 'piso', 'closets', 'chimenea',
       'permite_mascotas', 'gimnasio', 'ascensor', 'conjunto_cerrado'],
      dtype='object')

In [14]:
df.to_csv('../data/interim/apartments.csv', index=False)

In [15]:
import session_info

session_info.show()